In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import datetime
import re

In [3]:
def get_next_page(driver):
    try:
        driver.find_element_by_class_name("ui_button.nav.next.primary").send_keys(Keys.ENTER)
        return True
    except:
        print('마지막 페이지입니다')
        return False

In [4]:
def convert_date(date):
    stay_date = date.split('숙박 날짜: ')
    stay_date = stay_date[1]

    # 20XX년 X월 형태의 str
    stay_date = stay_date.replace('년', '').replace(' ', '').replace('월', '')
    dt = datetime.datetime.strptime(stay_date, '%Y%m')
    dt = dt.date() #20XX-XX-01 형태의 date 객체
    return dt

In [23]:
# convert_date 실행 예시
convert_date('숙박 날짜: 2020년 11월')

datetime.date(2020, 11, 1)

In [5]:
def get_data(driver):
    iteration = True
    df = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])
    try:
        time.sleep(20)
        #WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        #                                                                    "div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu")))
        # 더보기 클릭
        driver.find_element_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu").click()

        time.sleep(5)
        # 숙박 날짜 가져오기
        dates = driver.find_elements_by_class_name("_34Xs-BQm")
        stay_dates = []
        for d in dates:
            stay_date = convert_date(d.text)
            stay_dates.append(stay_date)
            # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
            if stay_date < datetime.date(2018,6,1): 
                iteration = False

        # 닉네임 가져오기
        nicknames = driver.find_elements_by_css_selector("div._310S4sqz > div > div._2fxQ4TOx > span > a")
        # 별점 가져오기
        rating_codes = driver.find_elements_by_css_selector("div.oETBfkHU > div._2UEC-y30 > div > span")
        scores = []
        for rc in rating_codes:
            cls_attr = str(rc.get_attribute("class")).split("ui_bubble_rating bubble_")
            score = int(cls_attr[1])
            scores.append(score)

        # 리뷰 가져오기
        reviews = driver.find_elements_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu > div.cPQsENeY > q > span")
        
        for i in range(len(nicknames)):
            df.loc[i] = [nicknames[i].text, scores[i], reviews[i].text, stay_dates[i]]
            #print(df.loc[i])

    except ElementClickInterceptedException as e:
        print(e)
    except StaleElementReferenceException as e:
        print(e)

    finally:
        return df, iteration

In [6]:
def hotel_review_crawling(url):
    # 데이터 수집
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)

    data = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])

    #첫페이지 시작
    #데이터 크롤링, df에 추가하기
    #다음페이지 클릭 -> disbaled 나오기 전까지 반복

    driver.get(url)
    driver.implicitly_wait(10)
    iteration = True
    page = 1
    while iteration:
        print(page, '번째 페이지')
        tmp_df, iteration = get_data(driver)
        data = pd.concat([data, tmp_df], axis=0, ignore_index=True)
        if iteration == False:
            break
        iteration = get_next_page(driver) 
        page += 1

    #data.to_csv('./hotel_review_test_sillastay_mapo.csv', encoding="utf-8-sig")
    
    return data

In [7]:
def preprocessing_data(data):
    # 개행문자 제거
    for idx, review in enumerate(data['review']):
        data['review'][idx] = review.replace('\n', ' ')
    # 날짜순으로 내림차순 정렬
    data.sort_values(by='stay_date', ascending=False, 
                     ignore_index=True, inplace=True)
    # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
    data = data[data['stay_date'] > datetime.date(2018,5,1)]
    # 특수문자, 이모지 제거
    ###
    return data

In [8]:
# 실제로 실행하는 코드
url = "https://www.tripadvisor.co.kr/Hotel_Review-g1119868-d10496451-Reviews-Shilla_Stay_Cheonan-Cheonan_Chungcheongnam_do.html"

data_sample = hotel_review_crawling(url)
data_sample

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지


,nickname,score,review,stay_date
0,현지 김,50,삼박자가 다 맞고 직원들도 다 친절하셔서 편안하게 쉬었다 갑니다\n제주도여행때 신라...,2021-07-01
1,wwawwa,50,천안에서는 여기 말고 갈때가 없습니다\n친절하고 깔끔하고 좋아요.\n조식도 좋고요....,2021-05-01
2,볼볼이,50,매우 청결하고 친절함\n작은소파와 책상도 있음\n비지니스 호텔 치고 방도 작지 않음...,2021-05-01
3,Susan Kim,30,전 높은층 디럭스 패밀리를 이용하였습니다. 비용은 비싸지 않아 좋았습니다\n다른도시...,2021-04-01
4,Jaylnsyy,50,"갑작스러운 일정으로 방문하게 되었는데\n침구도 포근하고, 대중교통도 편리하고 좋았습...",2020-08-01
5,Luna,50,친구들과 생일파티하려고 알아보다가 근처여서 예약하고 1박투숙하게 되었어요!! 주말에...,2020-07-01
6,Haru Lee,50,가족들이랑 다른지역가다가 천안에 들린김에 자고가게 되었어요 고속도로 톨게이트에서도 ...,2020-06-01
7,Ej kang,50,주변에 딱히 괜찮은 호텔이 없어서 이번에 호캉스 즐기러 가까운 천안 방문했습니다. ...,2020-05-01
8,jang junhyuk,50,부모님이 천안으로 가신후 아내랑 아이 데리고 갈때는 괜찮은데 재우고 싶어서 이곳저곳...,2020-05-01
9,shanxii,50,요즘 코로나 때문에 걱정 많이 했는데 ㅜㅜ..\n입구부터 한명 한명씩 열체크도 하고...,2020-05-01


In [9]:
# 기본 전처리 전 원본 데이터 저장(유실방지)
data_sample.to_csv('./data/hotel_review_shillastay_cheonan_origin.csv', encoding="utf-8-sig")

In [11]:
data = preprocessing_data(data_sample)

In [12]:
data

,nickname,score,review,stay_date
0,현지 김,50,삼박자가 다 맞고 직원들도 다 친절하셔서 편안하게 쉬었다 갑니다 제주도여행때 신라스...,2021-07-01
1,wwawwa,50,천안에서는 여기 말고 갈때가 없습니다 친절하고 깔끔하고 좋아요. 조식도 좋고요.. ...,2021-05-01
2,볼볼이,50,매우 청결하고 친절함 작은소파와 책상도 있음 비지니스 호텔 치고 방도 작지 않음 아...,2021-05-01
3,Susan Kim,30,전 높은층 디럭스 패밀리를 이용하였습니다. 비용은 비싸지 않아 좋았습니다 다른도시에...,2021-04-01
4,Jaylnsyy,50,"갑작스러운 일정으로 방문하게 되었는데 침구도 포근하고, 대중교통도 편리하고 좋았습니...",2020-08-01
5,Luna,50,친구들과 생일파티하려고 알아보다가 근처여서 예약하고 1박투숙하게 되었어요!! 주말에...,2020-07-01
6,Haru Lee,50,가족들이랑 다른지역가다가 천안에 들린김에 자고가게 되었어요 고속도로 톨게이트에서도 ...,2020-06-01
7,Ej kang,50,주변에 딱히 괜찮은 호텔이 없어서 이번에 호캉스 즐기러 가까운 천안 방문했습니다. ...,2020-05-01
8,jang junhyuk,50,부모님이 천안으로 가신후 아내랑 아이 데리고 갈때는 괜찮은데 재우고 싶어서 이곳저곳...,2020-05-01
9,shanxii,50,요즘 코로나 때문에 걱정 많이 했는데 ㅜㅜ.. 입구부터 한명 한명씩 열체크도 하고 ...,2020-05-01


In [13]:
# csv 저장을 위한 date to str convert
data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))
# 기본 전처리후 데이터 csv 파일로 저장 
#data.to_csv('./data/hotel_review_shillastay_yeoksam.csv', encoding="utf-8-sig")

<ipython-input-13-7183346d9643>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))


In [14]:
# 리뷰 내 이모지 제거
data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))
#data.to_csv('./data/hotel_review_shillastay_yeoksam_del_emoji.csv', encoding="utf-8-sig")

<ipython-input-14-b71e843350f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))


In [15]:
# 리뷰 내 특수문자 제거
test_data = data
special = re.compile(r'[^ A-Za-z0-9가-힣+]')
test_data['review'] = [special.sub('', s) for s in test_data['review']]

# 특수문자 제거 후 파일 저장
test_data.to_csv('./data/hotel_review_shillastay_cheonan_final.csv', encoding="utf-8-sig")

<ipython-input-15-fdc3440bfbaa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['review'] = [special.sub('', s) for s in test_data['review']]


In [16]:
test_data

,nickname,score,review,stay_date
0,현지 김,50,삼박자가 다 맞고 직원들도 다 친절하셔서 편안하게 쉬었다 갑니다 제주도여행때 신라스...,2021-07-01
1,wwawwa,50,천안에서는 여기 말고 갈때가 없습니다 친절하고 깔끔하고 좋아요 조식도 좋고요 접근성...,2021-05-01
2,볼볼이,50,매우 청결하고 친절함 작은소파와 책상도 있음 비지니스 호텔 치고 방도 작지 않음 아...,2021-05-01
3,Susan Kim,30,전 높은층 디럭스 패밀리를 이용하였습니다 비용은 비싸지 않아 좋았습니다 다른도시에서...,2021-04-01
4,Jaylnsyy,50,갑작스러운 일정으로 방문하게 되었는데 침구도 포근하고 대중교통도 편리하고 좋았습니다...,2020-08-01
5,Luna,50,친구들과 생일파티하려고 알아보다가 근처여서 예약하고 1박투숙하게 되었어요 주말에 투...,2020-07-01
6,Haru Lee,50,가족들이랑 다른지역가다가 천안에 들린김에 자고가게 되었어요 고속도로 톨게이트에서도 ...,2020-06-01
7,Ej kang,50,주변에 딱히 괜찮은 호텔이 없어서 이번에 호캉스 즐기러 가까운 천안 방문했습니다 호...,2020-05-01
8,jang junhyuk,50,부모님이 천안으로 가신후 아내랑 아이 데리고 갈때는 괜찮은데 재우고 싶어서 이곳저곳...,2020-05-01
9,shanxii,50,요즘 코로나 때문에 걱정 많이 했는데 입구부터 한명 한명씩 열체크도 하고 룸 도 ...,2020-05-01
